In [4]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
mainDF = pd.read_csv('rawData.csv')
finalDF = pd.read_csv('finalLadder.csv')

# Data Cleaning

In [6]:
mainDF['Attendance'] = mainDF['Attendance'].str.replace(',', '', regex=True)  # Remove commas
mainDF['Attendance'] = pd.to_numeric(mainDF['Attendance'], errors='coerce') 

## Data Encoding
Label Encoding is used for Venues and Teams since one-hot encoding will introduce a lot of empty tables and space. However, drawback is that ML model might misinterpret the venues and teams as having an order. Label encoding could assign large numbers to certain categories, making it harder for some algorithms to interpret the values

In [7]:
for column in mainDF.columns:
    print(column)
    print(type(mainDF[column][0]))

GameId
<class 'str'>
Year
<class 'numpy.int64'>
Round
<class 'numpy.int64'>
Date
<class 'str'>
MaxTemp
<class 'numpy.float64'>
MinTemp
<class 'numpy.float64'>
Rainfall
<class 'numpy.float64'>
Venue
<class 'str'>
StartTime
<class 'str'>
Attendance
<class 'numpy.int64'>
HomeTeam
<class 'str'>
HomeTeamScoreQT
<class 'numpy.float64'>
HomeTeamScoreHT
<class 'numpy.float64'>
HomeTeamScore3QT
<class 'numpy.float64'>
HomeTeamScoreFT
<class 'numpy.float64'>
HomeTeamScore
<class 'numpy.int64'>
AwayTeam
<class 'str'>
AwayTeamScoreQT
<class 'numpy.float64'>
AwayTeamScoreHT
<class 'numpy.float64'>
AwayTeamScore3QT
<class 'numpy.float64'>
AwayTeamScoreFT
<class 'numpy.float64'>
AwayTeamScore
<class 'numpy.int64'>
homePosition
<class 'numpy.float64'>
homePoints
<class 'numpy.float64'>
homePercentage
<class 'numpy.float64'>
awayPosition
<class 'numpy.float64'>
awayPoints
<class 'numpy.float64'>
awayPercentage
<class 'numpy.float64'>


In [8]:
print(mainDF['HomeTeam'].unique())
print(mainDF['Venue'].unique())

['Greater Western Sydney' 'Richmond' 'Hawthorn' 'Melbourne' 'Gold Coast'
 'Fremantle' 'North Melbourne' 'Western Bulldogs' 'Port Adelaide'
 'Brisbane Lions' 'Essendon' 'Sydney' 'West Coast' 'Adelaide'
 'Collingwood' 'St Kilda' 'Geelong' 'Carlton']
['Stadium Australia' 'M.C.G.' 'Carrara' 'Subiaco' 'Docklands'
 'Football Park' 'Gabba' 'S.C.G.' 'Bellerive Oval' 'Blacktown'
 'Kardinia Park' 'Manuka Oval' 'York Park' 'Marrara Oval'
 'Sydney Showground' "Cazaly's Stadium" 'Wellington' 'Adelaide Oval'
 'Traeger Park' 'Jiangwan Stadium' 'Eureka Stadium' 'Perth Stadium'
 'Riverway Stadium' 'Norwood Oval' 'Summit Sports Park']


In [9]:
team_name_mapping = {
    'Adelaide': 1,
    'Brisbane Lions': 2,
    'Carlton': 3,
    'Collingwood': 4,
    'Essendon': 5,
    'Fremantle': 6,
    'Geelong': 7,
    'Gold Coast': 8,
    'Greater Western Sydney': 9,
    'Hawthorn': 10,
    'Melbourne': 11,
    'North Melbourne': 12,
    'Port Adelaide': 13,
    'Richmond': 14,
    'St Kilda': 15,
    'Sydney': 16,
    'West Coast': 17,
    'Western Bulldogs': 18
}

mainDF['HomeTeam'] = mainDF['HomeTeam'].map(team_name_mapping)
mainDF['AwayTeam'] = mainDF['AwayTeam'].map(team_name_mapping)

finalDF['Team'] = finalDF['Team'].map(team_name_mapping)

finalDF

,Position,Team,Played,Points,Percentage,Round,Year
0,1,10,22,68,154.6,23,2012
1,2,1,22,68,132.5,23,2012
2,3,16,22,64,140.6,23,2012
3,4,4,22,64,116.5,23,2012
4,5,17,22,60,124.2,23,2012
...,...,...,...,...,...,...,...
229,14,11,23,44,98.5,25,2024
230,15,1,23,34,99.1,25,2024
231,16,17,23,20,68.1,25,2024
232,17,12,23,12,63.5,25,2024


In [10]:
# Removing stadiums which have low capacity or low usage
stadiums_to_remove = [
    'Bellerive Oval', 'Manuka Oval', 'Stadium Australia', 'Marrara Oval', 
    "Cazaly's Stadium", 'Eureka Stadium', 'Traeger Park', 'Wellington', 
    'Jiangwan Stadium', 'Norwood Oval', 'Blacktown', 'Riverway Stadium', 
    'Summit Sports Park'
]


mainDF = mainDF[~mainDF['Venue'].isin(stadiums_to_remove)]

venue_name_mapping = {
    'M.C.G.': 1, 'Carrara': 2, 'Subiaco': 3, 'Docklands': 4, 
    'Football Park': 5, 'Gabba': 6, 'S.C.G.': 7, 'Kardinia Park': 8, 
    'York Park': 9, 'Sydney Showground': 10, 'Adelaide Oval': 11, 
    'Perth Stadium': 12
}

mainDF['Venue'] = mainDF['Venue'].map(venue_name_mapping)

In [11]:
# Dealing with Datetime Features
mainDF['Date'] = pd.to_datetime(mainDF['Date'], errors='coerce')

#Create Day Variable
mainDF['Day'] = mainDF['Date'].dt.dayofweek  # Extract the day of the week (Monday=0, Sunday=6)

#Create Month Variable
mainDF['Month'] = mainDF['Date'].dt.month

# Assuming StartTime is in 12-hour clock format (e.g., '7:20 PM')
mainDF['StartTime'] = pd.to_datetime(mainDF['StartTime'], format='%I:%M %p', errors='coerce')

# If StartTime is in 24-hour clock format (e.g., '19:20')
# mainDF['StartTime'] = pd.to_datetime(mainDF['StartTime'], format='%H:%M', errors='coerce')

# Extract minutes since midnight
mainDF['MinutesSinceMidnight'] = mainDF['StartTime'].dt.hour * 60 + mainDF['StartTime'].dt.minute

#Holiday/Seasonal Indicators


In [12]:

#Dealing with Null Values
mainDF[mainDF.isnull().any(axis=1)]


# Create three new columns in mainDF: finalPosition, finalPoints, and finalPercentage
mainDF = mainDF.merge(
    finalDF[['Year', 'Team', 'Position', 'Points', 'Percentage']],
    left_on=['Year', 'HomeTeam'],
    right_on=['Year', 'Team'],
    how='left'
).rename(columns={'Position': 'finalPosition', 'Points': 'finalPoints', 'Percentage': 'finalPercentage'}) \
 .drop(columns=['Team'])


# Create three new columns in mainDF: finalPosition, finalPoints, and finalPercentage
mainDF = mainDF.merge(
    finalDF[['Year', 'Team', 'Position', 'Points', 'Percentage']],
    left_on=['Year', 'AwayTeam'],
    right_on=['Year', 'Team'],
    how='left'
).rename(columns={'Position': 'finalAPosition', 'Points': 'finalAPoints', 'Percentage': 'finalAPercentage'}) \
 .drop(columns=['Team'])


In [13]:
# Forward-fill NaN values within each HomeTeam group
mainDF['homePosition'] = mainDF.groupby('HomeTeam')['homePosition'].ffill()
mainDF['homePercentage'] = mainDF.groupby('HomeTeam')['homePercentage'].ffill()
mainDF['homePoints'] = mainDF.groupby('HomeTeam')['homePoints'].ffill()

mainDF['awayPosition'] = mainDF.groupby('AwayTeam')['awayPosition'].ffill()
mainDF['awayPercentage'] = mainDF.groupby('AwayTeam')['awayPercentage'].ffill()
mainDF['awayPoints'] = mainDF.groupby('AwayTeam')['awayPoints'].ffill()

# Removing Outliers

In [14]:
#Removing Years affected by COVID-19
mainDF = mainDF[(mainDF.Year != 2020)& (mainDF.Year != 2021)]

In [15]:
mainDF.to_csv('proFullData.csv', index=False)

In [16]:
# Create a new DataFrame for Home and Away Season
HADF = mainDF[mainDF.Round > 0]
HADF

,GameId,Year,Round,Date,MaxTemp,MinTemp,Rainfall,Venue,StartTime,Attendance,...,awayPercentage,Day,Month,MinutesSinceMidnight,finalPosition,finalPoints,finalPercentage,finalAPosition,finalAPoints,finalAPercentage
0,2012R0102,2012,1,2012-03-29,25.7,9.7,0.0,1,1900-01-01 19:45:00,78285,...,154.3,3,3,1185,12,42,111.6,10,44,108.0
1,2012R0103,2012,1,2012-03-30,27.4,9.7,0.0,1,1900-01-01 19:50:00,78466,...,83.9,4,3,1190,1,68,154.6,4,64,116.5
2,2012R0104,2012,1,2012-03-31,29.1,15.1,0.6,1,1900-01-01 13:45:00,33473,...,152.6,5,3,825,16,16,67.5,13,40,91.0
3,2012R0105,2012,1,2012-03-31,28.2,19.7,0.0,2,1900-01-01 15:45:00,12790,...,201.5,5,3,945,17,12,60.8,2,68,132.5
4,2012R0106,2012,1,2012-03-31,24.0,9.0,0.0,3,1900-01-01 16:45:00,34601,...,96.2,5,3,1005,7,56,115.7,6,60,117.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2284,2023R2405,2023,24,2023-08-26,13.9,9.4,0.0,8,1900-01-01 19:25:00,20533,...,108.7,5,8,1165,12,42,112.6,9,48,108.7
2285,2023R2406,2023,24,2023-08-26,24.3,9.5,0.0,12,1900-01-01 18:10:00,47027,...,116.8,5,8,1090,18,12,53.0,10,44,116.8
2286,2023R2407,2023,24,2023-08-27,16.0,9.1,0.0,11,1900-01-01 12:00:00,39860,...,93.6,6,8,720,3,68,112.7,13,42,93.6
2287,2023R2408,2023,24,2023-08-27,20.8,9.8,0.0,7,1900-01-01 15:20:00,41753,...,125.2,6,8,920,8,50,110.0,4,64,125.2


In [18]:
HADF.to_csv('proH&AData.csv', index=False)